<a href="https://colab.research.google.com/github/myungjinma/Python_Study/blob/main/0823_73%EC%9D%BC%EC%B0%A8ML(pygame%2C_gym%5Btoy_text%5D%2C_%EA%B3%84%EB%8B%A8%EB%AA%A8%ED%98%95%2C_%EC%83%81%ED%83%9C%EC%A0%84%EC%9D%B4_%ED%99%95%EB%A5%A0%EB%B6%84%ED%8F%AC%EC%99%80_%EC%A0%95%EC%B1%85%2C_%EC%B5%9C%EC%A0%81_%EC%A0%95%EC%B1%85%2C%EA%B0%80%EC%B9%98%ED%95%A8%EC%88%98%2C_%EB%B2%A8%EB%A7%8C_%EA%B8%B0%EB%8C%80%EB%B0%A9%EC%A0%95%EC%8B%9D).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install gym[toy_text]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 18.3 MB 83 kB/s 
  Attempting uninstall: pygame
    Found existing installation: pygame 2.1.2
    Uninstalling pygame-2.1.2:
      Successfully uninstalled pygame-2.1.2


In [4]:
import os
os.environ["SDL_VIDEODRIVER"] = "dummy"

In [5]:
import gym

In [6]:
env = gym.make("FrozenLake-v1", is_slippery=False)

/usr/local/lib/python3.7/dist-packages/gym/core.py:330: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."


In [7]:
env.observation_space

Discrete(16)

In [8]:
env.action_space

Discrete(4)

In [9]:
n_trial=20

In [10]:
env.reset()
episode=[]

In [11]:
for i in range(n_trial):
    action = env.action_space.sample()
    obs, reward, done, info = env.step(action)
    episode.append([action, reward, obs])
    env.render()
    if done:
        break
print('-------[행동, 보상, 상태]--------')
print(episode)
env.close()

/usr/local/lib/python3.7/dist-packages/gym/core.py:58: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  "You are calling render method, "


-------[행동, 보상, 상태]--------
[[0, 0.0, 0], [1, 0.0, 4], [0, 0.0, 4], [2, 0.0, 5]]


#계산 모형

마르코프 결정 과정 : 행동, 상태변화, 보상을 정의 하고 정책(최적 정책)을 찾는 일을 지원하는 수학적인 틀을 얘기한다.

마르코프 결정 과정의 핵심적인 문제는 의사결정자의 의사결정 정책(policy)

$$
π
$$
를 구하는 것이다.

#상태 전이 확률 분포와 정책

최적 정책을 알아내자



#9.2.2 최적 정책

정책은 현재 상태에서 어떤 행동을 취할지를 결정하는 확률 분포이다.

학습 알고리즘이 해야 할 일 : 누적 보상을 최대화하는 "최적 정책"을 알아내야 함

즉, P(a|s) 는 상태 s에서 행동 a가 일어날 확률이다. 그러므로 정책은 확률 분포로 표현 된다. 이 말을 이해하기 위해 FrozenLake의 예제를 보자

그렇다면 누적 보상이 최대가 되는 각 상태에서의 확률을 계산하기위해서는 어떻게 해야 하는 가? => 가치 함수를 이용한다.

#가치 함수

: 상태 s에서 정책이 얼마나 많은 보상을 제공할지를 계산하는 함수

$$
v_π(s),\quad \forall s ∈ \mathcal{S} \quad \quad \quad (9.4)
$$

* 최적 정책
: 가치함수를 최대화 하는 정책

$$π = aragmaru_π(s),\quad \forall s ∈ \mathcal{S} \quad \quad \quad (9.5)
$$

그러나 가치 함수를 어떻게 계산하는지는 제공하고 있지 않다.


#가치함수의 계산

$$
v_π(s) = E(r|s) = \sum_{s에서 출발하는 모든 에피소드 z} P(z)r(z), \quad \forall s ∈ \mathcal{S} \quad \quad \quad (9.6)
$$

가치 함수는 상태 s에서 발생할 수 있는 누적 보상액 r의 기댓값이다.

r의 기댓값은 s에서 출발하는 모든 에피소드에 대해 발생확률 p(z)와 누적 보상액 r(z)의 곱의 합이다.

가치 함수를 계산 하는 방법 (Pi_2 기준 <- Pi_1 정책에서는 0.5 였으므로 Pi_2가 Pi_1보다 최적화 되어 있다.)


$$
v_π(10) = E(z_2)r(z_2) = P(1|10)P(2|14)r(z_2) = 1*1*1 = 1
$$

#벨만 기대방정식

$$v{\pi} (s) = \sum{a \in \mathcal{A} (s)}P(a|s)(r+v_{\pi}(s')), \quad \forall s ∈ \mathcal{S} \quad \quad \quad (9.7)
$$

#할인율을 적용한 누적 보상액

gamma = 0 이면 현재 보상액만 고려한게 되어 근시안적인 추정

gamma=1 이면 미래에 발생하는 모든 보상을 같은 가중치로 계산

에피소드가 무한일 경우에 할인율이 없으면 누적 보상액이 무한대가 되므로 gamma는 1보다 작아야 한다.

#상태 가치 함수와 벨만기대방정식

보상의 할인율 gamma를 적용한 벨만기대방정식은 상태 가치 함수가 된다.

$$v{\pi} (s) = \sum{a \in \mathcal{A} (s)}P(a|s)(r+γv_{\pi}(s')), \forall s ∈ \mathcal{S} \quad \quad \quad (9.9)
$$